In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "WordCount")

22/03/01 02:00:19 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/01 02:00:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/01 02:00:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/01 02:00:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/01 02:00:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/01 02:00:22 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/03/01 02:00:22 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
fileRdd = sc.textFile("hdfs://localhost:9000/words.txt")

fileRdd.count()

6

In [6]:
fileRdd.collect()

['   spark kafka  ',
 ' kafka   spark pyspark ',
 '                ',
 'spark',
 '',
 'APACHE Kafka APache SParK ']

In [7]:
# RDD Lineage

lowerCaseRdd = fileRdd.map (lambda line: line.strip().lower())

lowerCaseRdd.collect()

['spark kafka',
 'kafka   spark pyspark',
 '',
 'spark',
 '',
 'apache kafka apache spark']

In [9]:
wordListRdd = lowerCaseRdd.map (lambda line: line.split(" "))
wordListRdd.collect()

[['spark', 'kafka'],
 ['kafka', '', '', 'spark', 'pyspark'],
 [''],
 ['spark'],
 [''],
 ['apache', 'kafka', 'apache', 'spark']]

In [11]:
# flatMap, remove the list, project element in the list as record
wordRdd = wordListRdd.flatMap(lambda elements: elements)
wordRdd.collect()

['spark',
 'kafka',
 'kafka',
 '',
 '',
 'spark',
 'pyspark',
 '',
 'spark',
 '',
 'apache',
 'kafka',
 'apache',
 'spark']

In [13]:
wordRdd =  wordRdd.filter (lambda word: word != "")
wordRdd.collect()

['spark',
 'kafka',
 'kafka',
 'spark',
 'pyspark',
 'spark',
 'apache',
 'kafka',
 'apache',
 'spark']

In [15]:
# convert word into (key,value) rdd (spark, 1) for reduceByKey
pairRdd = wordRdd.map (lambda word: (word, 1))
pairRdd.collect()

[('spark', 1),
 ('kafka', 1),
 ('kafka', 1),
 ('spark', 1),
 ('pyspark', 1),
 ('spark', 1),
 ('apache', 1),
 ('kafka', 1),
 ('apache', 1),
 ('spark', 1)]

In [16]:
# get word count using reduceByKey
wordCountRdd = pairRdd.reduceByKey(lambda acc, value: acc + value)
wordCountRdd.collect()

[('spark', 4), ('kafka', 3), ('pyspark', 1), ('apache', 2)]

In [17]:
wordCountRdd.getNumPartitions()

1

In [18]:
# write the result into text file in hdfs
# saveAsTextFile is an ACTION Method
# word-count-results1 is a folder, inside we will shall partition files

wordCountRdd.saveAsTextFile ("hdfs://localhost:9000/word-count-results1")

In [19]:
# hdfs dfs -ls /word-count-results1
#  _SUCCESS 0 bytes , to state that last operation successfuly stored
# part-00000 - partition files 
# note the partition file name, part-00000 or other file name 
# hdfs dfs -cat /word-count-results1/part-00000

# use hdfs web ui  http://localhost:50070/

In [20]:
# saveAsTextFile with two partitioned data
wordCountRdd.repartition(2)\
            .saveAsTextFile("hdfs://localhost:9000/word-count-results2")

In [ ]:
# hdfs dfs -ls /word-count-results2
# hdfs dfs -cat /word-count-results2/part-00000
# hdfs dfs -cat /word-count-results2/part-00001